**Week 3 Part 3 (Part 1 and 2 included)**

**1. Importing Libraries**

In [1]:
import numpy as np # library to handle data in a vectorized manner


import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


**2. Scraping the table data from wikipedia page using BeautifulSoup and creating a dataframe with only columns**

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text,"html.parser")

table = soup.find('table',{'class':'wikitable sortable'}).tbody

rows = table.find_all('tr')

columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df = pd.DataFrame(columns=columns)

In [3]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

**3. Reading the row data from table rows and appending it into the df dataframe.**

In [4]:
df.head()

,PostalCode,Borough,Neighbourhood


In [5]:
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    if tds[1].text != 'Not assigned':
            if (tds[2].text.replace('\n','') == 'Not assigned'):
                values = [tds[0].text,tds[1].text,tds[1].text]
            else :   
                values = [tds[0].text,tds[1].text,tds[2].text.replace('\n','')]
            df = df.append(pd.Series(values,index=columns),ignore_index=True)
    else:
        values = [td.text.replace('\n','') for td in tds]

In [6]:
df.head()

,PostalCode,Borough,Neighbourhood,Postcode
0,NaN,North York,Parkwoods,M3A
1,NaN,North York,Victoria Village,M4A
2,NaN,Downtown Toronto,Harbourfront,M5A
3,NaN,Downtown Toronto,Regent Park,M5A
4,NaN,North York,Lawrence Heights,M6A


**4. Deleting junk column 'PostalCode', renaming PostCode column to 'PostalCode' and re-positioning the column position**

In [7]:
df.drop(['PostalCode'],axis=1,inplace=True)
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['Borough', 'Neighbourhood', 'PostalCode'], dtype='object')

In [8]:
df = df[['PostalCode','Borough','Neighbourhood']]
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**5. Groupby Postcode and Borough and then join by Neighborhood so that there is only row per Postcode and Borough**

In [9]:
df_grouped = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda group: ','.join(group))
df_grouped.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df_grouped.shape

(103, 3)

**6. Reading the Toronto PostalCode coordinates form the csv file**

In [11]:
!wget -q -O 'toronto.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [12]:
toronto_coordinates = pd.read_csv('toronto.csv')
toronto_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**7. Renaming Postal Code to PostalCode for merging df_grouped and toronto_coordinates dataframes**

In [13]:
toronto_coordinates.rename(columns={"Postal Code":"PostalCode"},inplace=True)
toronto_coordinates.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [14]:
df_merge = pd.merge(df_grouped,toronto_coordinates, on = 'PostalCode')
df_merge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
df_merge.shape

(103, 5)

**8. Let's get the geographical coordinates of Toronto**

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**9.Create a map of Toronto with neighborhoods superimposed on top.**

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**10.Filterting boroughs with the word Toronto**

In [19]:
# filter borough names that contain the word Toronto
borough_names = list(df_merge.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)

# create a new DataFrame with only boroughs that contain the word Toronto
df_merge = df_merge[df_merge['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_merge.shape)
df_merge.head()

(38, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**11.Define Foursquare Credentials and Version**

In [22]:
CLIENT_ID = 'QBEOHTMYDSZGNPWZNKIMGAHL1BVY5KVZBJGFUPM3UDPN2JAF' # your Foursquare ID
CLIENT_SECRET = 'Q142X5G20QH1QKQEWWXLLF1KZDH4HD0LQVTNCQZH3VYEXJLL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QBEOHTMYDSZGNPWZNKIMGAHL1BVY5KVZBJGFUPM3UDPN2JAF
CLIENT_SECRET:Q142X5G20QH1QKQEWWXLLF1KZDH4HD0LQVTNCQZH3VYEXJLL


**12.Now, let's get the top 500 venues that are within a radius of 1000 meters.**

In [25]:
LIMIT =500
radius = 1000
venues = []

for lat, long, post, borough, neighbourhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['PostalCode'], df_merge['Borough'], df_merge['Neighbourhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'Borough Lat', 'Borough Long', 'Venue Name', 'Venue Lat', 'Venue Long', 'Venue Category']

print(venues_df.shape)
venues_df.head()

(3084, 9)


,PostalCode,Borough,Neighbourhood,Borough Lat,Borough Long,Venue Name,Venue Lat,Venue Long,Venue Category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop


In [27]:
venues_df.groupby(["PostalCode", "Borough","Neighbourhood"]).count()

,,,Borough Lat,Borough Long,Venue Name,Venue Lat,Venue Long,Venue Category
PostalCode,Borough,Neighbourhood,,,,,,
M4E,East Toronto,The Beaches,80,80,80,80,80,80
M4K,East Toronto,"The Danforth West,Riverdale",100,100,100,100,100,100
M4L,East Toronto,"The Beaches West,India Bazaar",78,78,78,78,78,78
M4M,East Toronto,Studio District,100,100,100,100,100,100
M4N,Central Toronto,Lawrence Park,9,9,9,9,9,9
M4P,Central Toronto,Davisville North,100,100,100,100,100,100
M4R,Central Toronto,North Toronto West,42,42,42,42,42,42
M4S,Central Toronto,Davisville,100,100,100,100,100,100
M4T,Central Toronto,"Moore Park,Summerhill East",61,61,61,61,61,61


#### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

There are 267 uniques categories.


In [29]:
venues_df['Venue Category'].unique()[:50]

array(['Vegetarian / Vegan Restaurant', 'Trail', 'Indie Movie Theater',
       'Gastropub', 'Ice Cream Shop', 'Bakery', 'Bagel Shop',
       'Health Food Store', 'Toy / Game Store', 'Coffee Shop',
       'Breakfast Spot', 'French Restaurant', 'Pub', 'Park', 'Beach',
       'Bar', 'Mexican Restaurant', 'Japanese Restaurant', 'Tea Room',
       'Liquor Store', 'Juice Bar', 'Diner', 'Nail Salon',
       'Caribbean Restaurant', 'Cupcake Shop', 'Bookstore',
       'Sandwich Place', 'Burger Joint', 'Jazz Club', 'Supermarket',
       'Chocolate Shop', 'Thai Restaurant', 'Ramen Restaurant',
       'Pharmacy', 'Indian Restaurant', 'Café', 'Asian Restaurant',
       'Pizza Place', 'Greek Restaurant', 'Beer Store', 'Bank',
       'Grocery Store', 'Electronics Store', 'Restaurant', 'Pet Store',
       'Shoe Store', 'Flower Shop', 'Church', "Women's Store",
       'Scenic Lookout'], dtype=object)

**13.Analyze each area**

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [31]:
toronto_onehot.shape

(3084, 268)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide,King,Richmond",0.00,0.000000,0.000000,0.030000,0.00,0.00,0.000000,0.00,0.010000,0.01,0.010000,0.020000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.00,0.02,

In [33]:
toronto_grouped.shape

(38, 268)

#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                Hotel  0.05
2                 Café  0.05
3              Theater  0.04
4  American Restaurant  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.08
1                Hotel  0.06
2                 Café  0.05
3             Beer Bar  0.04
4  Japanese Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.06
2              Restaurant  0.04
3  Furniture / Home Store  0.04
4                     Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0         Park  0.08
1      Brewery  0.06
2  Coffee Shop  0.06
3  Pizza Place  0.06
4    Pet Store  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0       Coffee Shop  0.13
1    

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Hotel,Café,Theater,Japanese Restaurant,American Restaurant,Sushi Restaurant,Bakery,Furniture / Home Store,Clothing Store
1,Berczy Park,Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar,Cocktail Bar,Bakery,Park,Restaurant,Gym
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Restaurant,Furniture / Home Store,Bar,Tibetan Restaurant,Bakery,Thrift / Vintage Store,Beer Bar,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Park,Pizza Place,Coffee Shop,Brewery,Pet Store,Sushi Restaurant,Italian Restaurant,Burrito Place,Snack Place,Breakfast Spot
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Café,Coffee Shop,Harbor / Marina,Dog Run,Garden,Scenic Lookout,Park,Track,Sculpture Garden,Dance Studio


**14. Cluster Neighbourhoods**

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 3, 1, 1, 4, 4, 1], dtype=int32)

In [39]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merge

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Pizza Place,Beach,Breakfast Spot,Bar,Japanese Restaurant,Tea Room,Bakery,Park
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Pizza Place,Italian Restaurant,Ice Cream Shop,Ramen Restaurant,Sandwich Place
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Indian Restaurant,Coffee Shop,Pizza Place,Beach,Café,Sandwich Place,Bakery,Brewery,Restaurant,Park
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Coffee Shop,Bar,American Restaurant,Café,Vietnamese Restaurant,Bakery,Brewery,Italian Restaurant,Diner,Arts & Crafts Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Coffee Shop,Trail,Bus Line,Park,Bookstore,College Quad,College Gym,Gym / Fitness Center,Café,Ethiopian Restaurant


Finally, let's visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**15. Examine the clusters**

**Cluster 1**

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Pub,Pizza Place,Beach,Breakfast Spot,Bar,Japanese Restaurant,Tea Room,Bakery,Park
1,East Toronto,0,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Pizza Place,Italian Restaurant,Ice Cream Shop,Ramen Restaurant,Sandwich Place
2,East Toronto,0,Indian Restaurant,Coffee Shop,Pizza Place,Beach,Café,Sandwich Place,Bakery,Brewery,Restaurant,Park
5,Central Toronto,0,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pizza Place,Café,Dessert Shop,Gym,Sushi Restaurant,Pharmacy,Movie Theater
6,Central Toronto,0,Italian Restaurant,Coffee Shop,Sporting Goods Shop,Café,Diner,Mexican Restaurant,Spa,Sushi Restaurant,Food & Drink Shop,Flower Shop
7,Central Toronto,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Pizza Place,Indian Restaurant,Dessert Shop,Gym,Fast Food Restaurant
8,Central Toronto,0,Coffee Shop,Italian Restaurant,Park,Gym,Grocery Store,Playground,Pizza Place,Thai Restaurant,Bank,Bagel Shop
9,Central Toronto,0,Coffee Shop,Italian Restaurant,Park,Sushi Restaurant,Pizza Place,Grocery Store,Thai Restaurant,Bagel Shop,Liquor Store,Gym
13,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Theater,Breakfast Spot,Park,Pub,Restaurant,Diner,Bakery
22,Central Toronto,0,Sushi Restaurant,Italian Restaurant,Pharmacy,Café,Bank,Coffee Shop,Pilates Studio,Gastropub,Clothing Store,Bakery


**Cluster 2**

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,1,Coffee Shop,Bank,Park,Grocery Store,Convenience Store,Playground,Office,Sandwich Place,Shopping Mall,Caribbean Restaurant
11,Downtown Toronto,1,Park,Restaurant,Diner,Coffee Shop,Gastropub,Café,Japanese Restaurant,Rock Club,Bistro,Beer Store
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Restaurant,Dance Studio,Diner,Burger Joint,Men's Store,Mediterranean Restaurant
14,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Middle Eastern Restaurant,Restaurant,Café,Italian Restaurant,Gastropub,Japanese Restaurant
15,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Gastropub,Seafood Restaurant,Cosmetics Shop,Breakfast Spot
16,Downtown Toronto,1,Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar,Cocktail Bar,Bakery,Park,Restaurant,Gym
17,Downtown Toronto,1,Coffee Shop,Ice Cream Shop,Café,Park,Ramen Restaurant,Vegetarian / Vegan Restaurant,Bubble Tea Shop,Italian Restaurant,Burger Joint,Bookstore
18,Downtown Toronto,1,Coffee Shop,Hotel,Café,Theater,Japanese Restaurant,American Restaurant,Sushi Restaurant,Bakery,Furniture / Home Store,Clothing Store
19,Downtown Toronto,1,Coffee Shop,Hotel,Café,Aquarium,Italian Restaurant,Brewery,Bakery,Scenic Lookout,Park,Japanese Restaurant
20,Downtown Toronto,1,Hotel,Coffee Shop,Café,Italian Restaurant,Steakhouse,American Restaurant,Restaurant,Concert Hall,Japanese Restaurant,Beer Bar


**Cluster 3**

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Coffee Shop,Trail,Bus Line,Park,Bookstore,College Quad,College Gym,Gym / Fitness Center,Café,Ethiopian Restaurant


**Cluster 4**

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,3,Café,Coffee Shop,Harbor / Marina,Dog Run,Garden,Scenic Lookout,Park,Track,Sculpture Garden,Dance Studio


**Cluster 5**

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Coffee Shop,Bar,American Restaurant,Café,Vietnamese Restaurant,Bakery,Brewery,Italian Restaurant,Diner,Arts & Crafts Store
25,Downtown Toronto,4,Café,Bakery,Vegetarian / Vegan Restaurant,Bar,Restaurant,Bookstore,Mexican Restaurant,Arts & Crafts Store,Dessert Shop,Pizza Place
26,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Coffee Shop,Art Gallery,Mexican Restaurant,Beer Bar,Yoga Studio,Caribbean Restaurant
30,Downtown Toronto,4,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Indian Restaurant,Cocktail Bar,Comedy Club,Park,Japanese Restaurant
32,West Toronto,4,Café,Bar,Bakery,Coffee Shop,Restaurant,Cocktail Bar,Italian Restaurant,Wine Bar,Asian Restaurant,Theater
33,West Toronto,4,Café,Coffee Shop,Restaurant,Furniture / Home Store,Bar,Tibetan Restaurant,Bakery,Thrift / Vintage Store,Beer Bar,Italian Restaurant
